Evaluation code for coordinates data(cnn_model_new3.keras)

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os

# Predefined angle data for each class (ensuring each list has 10 angles)
class_angles = {
    'hachijidachijodanyoko': [
        [178.6704642, 121.4324436, 178.6704642, 121.4324436, 176.2882244, 176.2882244, 179.0472569, 155.9306886, 174.7958128, 0],
        [74.73670094, 179.6333059, 74.73670094, 179.6333059, 179.2610734, 179.2610734, 177.7071013, 157.4409467, 146.1440207, 0]
    ],
    'sanchindachiageuke': [
        [117.6311735, 177.122714, 117.6311735, 177.122714, 172.4470531, 172.4470531, 179.84531, 178.0124464, 154.3915227, 0],
        [88.35833, 130.8359801, 88.35833, 130.8359801, 178.8655222, 178.8655222, 179.2608471, 159.6653805, 172.8783815, 0]
    ],
    'sanchindachijodantsuki': [
        [79.67428505, 117.6143462, 79.67428505, 117.6143462, 175.4425771, 175.4425771, 179.9083308, 179.6819875, 162.7090183, 0],
        [132.2753167, 61.93098373, 132.2753167, 61.93098373, 177.3881788, 177.3881788, 177.6381035, 156.7447632, 179.4797426, 0]
    ],
    'sanchindachisotouke': [
        [11.47716376, 114.5169405, 11.47716376, 114.5169405, 176.3834111, 176.3834111, 175.592297, 179.2147185, 163.3744971, 0],
        [87.20938557, 16.70051146, 87.20938557, 16.70051146, 178.365065, 178.365065, 174.4879716, 162.4809264, 176.8140283, 0]
    ],
    'shikodachigedanbarai': [
        [176.4795284, 153.3314441, 176.4795284, 153.3314441, 107.3410318, 107.3410318, 97.62956052, 153.8734055, 156.7703905, 0],
        [147.1951634, 175.074177, 147.1951634, 175.074177, 103.3349196, 103.3349196, 128.5222031, 157.1361985, 145.8509965, 0]
    ],
    'sotoukemaegeri': [
        [141.6715664, 12.61353582, 141.6715664, 12.61353582, 177.0001987, 177.0001987, 175.727286, 164.0403825, 160.4981336, 0],
        [15.39189415, 84.51329129, 15.39189415, 84.51329129, 174.1109382, 174.1109382, 158.9654381, 178.2361718, 159.3976762, 0]
    ],
    'zenkutsudachiawasetsuki': [
        [115.8580351, 130.7511613, 115.8580351, 130.7511613, 171.9446945, 171.9446945, 172.6747746, 144.105041, 178.9400008, 0]
    ],
    'zenkutsudachichudantsuki': [
        [115.8580351, 130.7511613, 115.8580351, 130.7511613, 171.9446945, 171.9446945, 172.6747746, 144.105041, 178.9400008, 0],
        [156.0357435, 61.09435279, 156.0357435, 61.09435279, 172.3895637, 172.3895637, 162.5085944, 157.2766185, 159.0072517, 0]
    ],
    'zenkutsudachiempiuke': [
        [5.537744135, 138.5696075, 5.537744135, 138.5696075, 176.3914556, 176.3914556, 173.9572285, 166.2690671, 157.8046774, 0],
        [156.0357435, 61.09435279, 156.0357435, 61.09435279, 172.3895637, 172.3895637, 162.5085944, 157.2766185, 159.0072517, 0]
    ]
}

# Mapping from class indexes to class names
class_index_to_name = {
    0: 'hachijidachijodanyoko',
    1: 'sanchindachiageuke',
    2: 'sanchindachijodantsuki',
    3: 'sanchindachisotouke',
    4: 'shikodachigedanbarai',
    5: 'sotoukemaegeri',
    6: 'zenkutsudachiawasetsuki',
    7: 'zenkutsudachichudantsuki',
    8: 'zenkutsudachiempiuke'
}

def load_model(model_path):
    model = tf.keras.models.load_model(model_path)
    return model

# Function to extract 3D keypoints using Mediapipe
def extract_keypoints(image):
    mp_pose = mp.solutions.pose
    with mp_pose.Pose(static_image_mode=True) as pose:
        image_resized = cv2.resize(image, (512, 384))
        results = pose.process(cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB))
        if not results.pose_landmarks:
            return None
        keypoints = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.pose_landmarks.landmark], dtype=np.float32)
    return keypoints

def normalize_keypoints(keypoints):
    # Normalizes keypoints to a range between 0 and 1
    min_val = np.min(keypoints, axis=0)
    max_val = np.max(keypoints, axis=0)
    normalized_keypoints = (keypoints - min_val) / (max_val - min_val + 1e-6)  # Small epsilon to prevent division by zero
    return normalized_keypoints

# Function to load and preprocess keypoints
def preprocess_keypoints(keypoints):
    # Flatten the keypoints to match model's expected input shape (None, 99)
    flattened_keypoints = keypoints.reshape(-1)  # Flatten from (33, 3) to (99,)
    return np.expand_dims(flattened_keypoints, axis=0)

# Function to classify pose using a loaded model and keypoints
def classify_pose(model, keypoints):
    # Preprocess keypoints to match the model's input shape
    reshaped_keypoints = preprocess_keypoints(keypoints)
    prediction = model.predict(reshaped_keypoints)
    class_index = np.argmax(prediction)
    return class_index  # 


def calculate_angle(a, b, c):
    # Calculate the angle between three points
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

def extract_angles(keypoints):
    if keypoints is None:
        return None
    
    LEFT_SHOULDER = mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value
    RIGHT_SHOULDER = mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value
    LEFT_ELBOW = mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value
    RIGHT_ELBOW = mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value
    LEFT_WRIST = mp.solutions.pose.PoseLandmark.LEFT_WRIST.value
    RIGHT_WRIST = mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value
    LEFT_HIP = mp.solutions.pose.PoseLandmark.LEFT_HIP.value
    RIGHT_HIP = mp.solutions.pose.PoseLandmark.RIGHT_HIP.value
    LEFT_KNEE = mp.solutions.pose.PoseLandmark.LEFT_KNEE.value
    RIGHT_KNEE = mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value
    LEFT_ANKLE = mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value
    RIGHT_ANKLE = mp.solutions.pose.PoseLandmark.RIGHT_ANKLE.value

    left_shoulder_angle = calculate_angle(keypoints[LEFT_ELBOW], keypoints[LEFT_SHOULDER], keypoints[LEFT_HIP])
    right_shoulder_angle = calculate_angle(keypoints[RIGHT_HIP], keypoints[RIGHT_SHOULDER], keypoints[RIGHT_ELBOW])
    left_elbow_angle = calculate_angle(keypoints[LEFT_SHOULDER], keypoints[LEFT_ELBOW], keypoints[LEFT_WRIST])
    right_elbow_angle = calculate_angle(keypoints[RIGHT_WRIST], keypoints[RIGHT_ELBOW], keypoints[RIGHT_SHOULDER])
    left_waist_angle = calculate_angle(keypoints[LEFT_KNEE], keypoints[LEFT_HIP], keypoints[LEFT_SHOULDER])
    right_waist_angle = calculate_angle(keypoints[RIGHT_SHOULDER], keypoints[RIGHT_HIP], keypoints[RIGHT_KNEE])
    left_knee_angle = calculate_angle(keypoints[LEFT_HIP], keypoints[LEFT_KNEE], keypoints[LEFT_ANKLE])
    right_knee_angle = calculate_angle(keypoints[RIGHT_ANKLE], keypoints[RIGHT_KNEE], keypoints[RIGHT_HIP])
    left_ankle_angle = calculate_angle(keypoints[LEFT_KNEE], keypoints[LEFT_ANKLE], keypoints[LEFT_HIP])
    right_ankle_angle = calculate_angle(keypoints[RIGHT_HIP], keypoints[RIGHT_ANKLE], keypoints[RIGHT_KNEE])
    
    angles = [
        left_shoulder_angle, right_shoulder_angle, 
        left_elbow_angle, right_elbow_angle, 
        left_waist_angle, right_waist_angle, 
        left_knee_angle, right_knee_angle, 
        left_ankle_angle, right_ankle_angle
    ]
    
    return angles

# Function to compare angles and calculate similarity
def compare_angles(extracted_angles, predefined_angles):
    similarities = []
    for angle_set in predefined_angles:
        diff = np.abs(np.array(extracted_angles) - np.array(angle_set))
        similarity = np.mean(1 - (diff / 180.0))
        similarities.append(similarity * 100)
    max_similarity = max(similarities)
    return max_similarity

# Function to load image
def load_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Error loading image")
    return image

def plot_3d_keypoints(keypoints, ax):
    x = keypoints[:, 0]
    y = keypoints[:, 1]
    z = keypoints[:, 2]
    ax.scatter(x, y, z)
    for i, point in enumerate(keypoints):
        ax.text(point[0], point[1], point[2], str(i))

def process_image(image_path, model):
    # Load and process the image
    image = load_image(image_path)
    
    # Extract keypoints using MediaPipe
    keypoints = extract_keypoints(image)
    if keypoints is None:
        print("No keypoints detected.")
        return None
    
    # Normalize the keypoints
    normalized_keypoints = normalize_keypoints(keypoints)
    
    # Classify the pose
    class_index = classify_pose(model, normalized_keypoints)
    class_name = class_index_to_name.get(class_index, "Unknown")
    print(f"Predicted Class: {class_name}")

    # Extract angles from the keypoints
    extracted_angles = extract_angles(keypoints)
    if extracted_angles is None:
        print("Angle extraction failed.")
        return None
    
    # Compare extracted angles with predefined class angles
    predefined_angles = class_angles.get(class_name, [])
    similarity = compare_angles(extracted_angles, predefined_angles)
    print(f"Similarity with {class_name} angles: {similarity:.2f}%")

    # Plot the 3D keypoints for visualization
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    plot_3d_keypoints(keypoints, ax)
    plt.title(f"3D Keypoints - {class_name}")
    plt.show()
    
    return class_name, similarity

# Example usage
model_path = 'cnn_model_new3.keras'
model = load_model(model_path)
image_path = 'test_folder/sanchinjodan2.png'
process_image(image_path, model)



Evaluation code for image data(cnn_model_augmented_dropout2.keras)

In [ ]:
import os
import tensorflow as tf
import numpy as np
import cv2
import mediapipe as mp
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import time
import tracemalloc

# Start time
start_time = time.time()


# Start memory tracking
tracemalloc.start()

# Load your .keras model
model = load_model('cnn_model_augmented_dropout2.keras')

# Define the image size your model expects
IMG_SIZE = (224, 224)

# Define the class names
class_names = ['hachijiDachi_jodanYoko', 'sanchinDachi_ageUke', 'sanchinDachi_jodanTsuki', 
               'sanchinDachi_sotoUke', 'shikoDachi_gedanBarai', 'sotoUke_maeGeri', 
               'zenkutsuDachi_awaseTsuki', 'zenkutsuDachi_chudanTsuki', 'zenkutsuDachi_empiUke']

# Predefined angle data for each class
class_angles = {
        'hachijiDachi_jodanYoko': [
        [178.6704642, 121.4324436, 178.6704642, 121.4324436, 176.2882244, 176.2882244, 179.0472569, 155.9306886, 174.7958128, 0],
        [74.73670094, 179.6333059, 74.73670094, 179.6333059, 179.2610734, 179.2610734, 177.7071013, 157.4409467, 146.1440207, 0]
    ],
    'sanchinDachi_ageUke': [
        [117.6311735, 177.122714, 117.6311735, 177.122714, 172.4470531, 172.4470531, 179.84531, 178.0124464, 154.3915227, 0],
        [88.35833, 130.8359801, 88.35833, 130.8359801, 178.8655222, 178.8655222, 179.2608471, 159.6653805, 172.8783815, 0]
    ],
    'sanchinDachi_jodanTsuki': [
        [79.67428505, 117.6143462, 79.67428505, 117.6143462, 175.4425771, 175.4425771, 179.9083308, 179.6819875, 162.7090183, 0],
        [132.2753167, 61.93098373, 132.2753167, 61.93098373, 177.3881788, 177.3881788, 177.6381035, 156.7447632, 179.4797426, 0]
    ],
    'sanchinDachi_sotoUke': [
        [11.47716376, 114.5169405, 11.47716376, 114.5169405, 176.3834111, 176.3834111, 175.592297, 179.2147185, 163.3744971, 0],
        [87.20938557, 16.70051146, 87.20938557, 16.70051146, 178.365065, 178.365065, 174.4879716, 162.4809264, 176.8140283, 0]
    ],
    'shikoDachi_gedanBarai': [
        [176.4795284, 153.3314441, 176.4795284, 153.3314441, 107.3410318, 107.3410318, 97.62956052, 153.8734055, 156.7703905, 0],
        [147.1951634, 175.074177, 147.1951634, 175.074177, 103.3349196, 103.3349196, 128.5222031, 157.1361985, 145.8509965, 0]
    ],
    'sotoUke_maeGeri': [
        [141.6715664, 12.61353582, 141.6715664, 12.61353582, 177.0001987, 177.0001987, 175.727286, 164.0403825, 160.4981336, 0],
        [15.39189415, 84.51329129, 15.39189415, 84.51329129, 174.1109382, 174.1109382, 158.9654381, 178.2361718, 159.3976762, 0]
    ],
    'zenkutsuDachi_awaseTsuki': [
        [115.8580351, 130.7511613, 115.8580351, 130.7511613, 171.9446945, 171.9446945, 172.6747746, 144.105041, 178.9400008, 0]
    ],
    'zenkutsuDachi_chudanTsuki': [
        [115.8580351, 130.7511613, 115.8580351, 130.7511613, 171.9446945, 171.9446945, 172.6747746, 144.105041, 178.9400008, 0],
        [156.0357435, 61.09435279, 156.0357435, 61.09435279, 172.3895637, 172.3895637, 162.5085944, 157.2766185, 159.0072517, 0]
    ],
    'zenkutsuDachi_empiUke': [
        [5.537744135, 138.5696075, 5.537744135, 138.5696075, 176.3914556, 176.3914556, 173.9572285, 166.2690671, 157.8046774, 0],
        [156.0357435, 61.09435279, 156.0357435, 61.09435279, 172.3895637, 172.3895637, 162.5085944, 157.2766185, 159.0072517, 0]
    ]
}

# Folder path containing images to classify
image_folder_path = 'C:/Users/WW/anaconda3/projects/11-11/test_folder'

# Function to preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to extract 3D keypoints using Mediapipe
def extract_keypoints(img_path):
    mp_pose = mp.solutions.pose
    image = cv2.imread(img_path)
    with mp_pose.Pose(static_image_mode=True) as pose:
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if not results.pose_landmarks:
            return None
        keypoints = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.pose_landmarks.landmark], dtype=np.float32)
    return keypoints

# Calculate angle between three points
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Extract angles from keypoints
def extract_angles(keypoints):
    if keypoints is None:
        return None
    
    LEFT_SHOULDER = mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value
    RIGHT_SHOULDER = mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value
    LEFT_ELBOW = mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value
    RIGHT_ELBOW = mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value
    LEFT_WRIST = mp.solutions.pose.PoseLandmark.LEFT_WRIST.value
    RIGHT_WRIST = mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value
    LEFT_HIP = mp.solutions.pose.PoseLandmark.LEFT_HIP.value
    RIGHT_HIP = mp.solutions.pose.PoseLandmark.RIGHT_HIP.value
    LEFT_KNEE = mp.solutions.pose.PoseLandmark.LEFT_KNEE.value
    RIGHT_KNEE = mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value
    LEFT_ANKLE = mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value
    RIGHT_ANKLE = mp.solutions.pose.PoseLandmark.RIGHT_ANKLE.value

    # Calculate angles between specific points
    left_shoulder_angle = calculate_angle(keypoints[LEFT_ELBOW], keypoints[LEFT_SHOULDER], keypoints[LEFT_HIP])
    right_shoulder_angle = calculate_angle(keypoints[RIGHT_HIP], keypoints[RIGHT_SHOULDER], keypoints[RIGHT_ELBOW])
    left_elbow_angle = calculate_angle(keypoints[LEFT_SHOULDER], keypoints[LEFT_ELBOW], keypoints[LEFT_WRIST])
    right_elbow_angle = calculate_angle(keypoints[RIGHT_WRIST], keypoints[RIGHT_ELBOW], keypoints[RIGHT_SHOULDER])
    left_waist_angle = calculate_angle(keypoints[LEFT_KNEE], keypoints[LEFT_HIP], keypoints[LEFT_SHOULDER])
    right_waist_angle = calculate_angle(keypoints[RIGHT_SHOULDER], keypoints[RIGHT_HIP], keypoints[RIGHT_KNEE])
    left_knee_angle = calculate_angle(keypoints[LEFT_HIP], keypoints[LEFT_KNEE], keypoints[LEFT_ANKLE])
    right_knee_angle = calculate_angle(keypoints[RIGHT_ANKLE], keypoints[RIGHT_KNEE], keypoints[RIGHT_HIP])
    left_ankle_angle = calculate_angle(keypoints[LEFT_KNEE], keypoints[LEFT_ANKLE], keypoints[LEFT_HIP])
    right_ankle_angle = calculate_angle(keypoints[RIGHT_HIP], keypoints[RIGHT_ANKLE], keypoints[RIGHT_KNEE])
    
    angles = [
        left_shoulder_angle, right_shoulder_angle, 
        left_elbow_angle, right_elbow_angle, 
        left_waist_angle, right_waist_angle, 
        left_knee_angle, right_knee_angle, 
        left_ankle_angle, right_ankle_angle
    ]
    
    return angles

# Function to compare angles and calculate similarity
def compare_angles(extracted_angles, predefined_angles):
    similarities = []
    for angle_set in predefined_angles:
        diff = np.abs(np.array(extracted_angles) - np.array(angle_set))
        similarity = np.mean(1 - (diff / 180.0))
        similarities.append(similarity * 100)
    max_similarity = max(similarities)
    return max_similarity

# Loop through all images in the folder and classify them
for img_name in os.listdir(image_folder_path):
    img_path = os.path.join(image_folder_path, img_name)
    
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_array = preprocess_image(img_path)
        
        # Predict the class and confidence
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions, axis=1)[0]
        confidence = predictions[0][predicted_class_index]
        
        # Get keypoints and calculate angles
        keypoints = extract_keypoints(img_path)
        extracted_angles = extract_angles(keypoints)
        
        if extracted_angles:
            print(f"Image: {img_name}")
            print(f"Predicted Class: {class_names[predicted_class_index]}")
            #print(f"Confidence Level: {confidence * 100:.2f}%")
            #print(f"Extracted Angles: {extracted_angles}")
            
            # Compare with predefined angles for the predicted class
            class_name = class_names[predicted_class_index]
            predefined_angles = class_angles.get(class_name, [])
            
            # Calculate similarity if predefined angles exist
            if predefined_angles:  # <-- Highlighted Change
                similarity = compare_angles(extracted_angles, predefined_angles)
                print(f"Similarity with predefined angles: {similarity:.2f}%\n")
            else:
                print(f"No predefined angles found for class '{class_name}'\n")
        else:
            print(f"Image: {img_name} - Could not extract angles\n")

end_time = time.time()
print(f"Execution Time: {end_time - start_time:.2f} seconds")
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
print(f"Current memory usage: {current / 10**6:.2f} MB")
print(f"Peak memory usage: {peak / 10**6:.2f} MB")